In [34]:
import nltk
import pandas as pd
import numpy as np
nltk.download("popular")
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import PCA
from sklearn import preprocessing
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.preprocessing import LabelEncoder
stop_words = list(stopwords.words('english'))

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /Users/manu/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /Users/manu/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /Users/manu/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /Users/manu/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /Users/manu/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /Users/manu/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!

In [35]:
df = pd.read_csv('./final60_amzn.csv')

In [36]:
df

,Unnamed: 0,text,sentiment,label,stock_time
0,0,nape summit week will also feature the annual ...,0.050660,-1.0,2017-12-07 20:00:00
1,1,from apples hugely anticipated iphone x to sam...,0.107128,1.0,2017-12-08 22:00:00
2,2,"they are not just the largest browser, but the...",0.034394,1.0,2017-12-12 02:00:00
3,3,the humanitarian crisis in the drc has placed ...,0.000000,-1.0,2017-12-12 22:30:00
4,4,cramer prefers finisar (nasdaq: fnsr ) after a...,0.035844,-1.0,2017-12-14 18:30:00
...,...,...,...,...,...
4247,4247,this report just confirmed apple's china probl...,0.013567,1.0,2019-02-01 22:30:00
4248,4248,apple's (nasdaq: aapl ) spectacular rise and f...,0.017423,-1.0,2019-02-01 23:00:00
4249,4249,"so, lets see what to expect from the entertain...",0.029321,-1.0,2019-02-01 23:30:00
4250,4250,"investorplace.com published on january 30, 20...",0.025605,-1.0,2019-02-02 00:00:00


In [37]:
# train with ngrams and sentiment
text = df['text'].fillna('')
sent = df['sentiment']
labels = df['label']

# generate ngrams = 2,2 or 2,3 with Count Vectorizer
count_vectorizer = CountVectorizer(max_df=0.8, min_df=0.02, stop_words=stop_words, ngram_range=(2,2))
X = count_vectorizer.fit_transform(text)

# use TfIdf to determine freq
tfidf = TfidfTransformer(smooth_idf=True, use_idf=True)
X = tfidf.fit_transform(X).toarray()

# use PCA to generate final text dataset
pca = PCA(n_components=3)
pca_comp = pca.fit_transform(X)

# join text and sentiment
finalXtrain = np.hstack((pca_comp, np.atleast_2d(sent).T))

# create train and test split
X_train, X_test, y_train, y_test = train_test_split(finalXtrain, labels, test_size=0.3, random_state=15)

# scale the data
minmaxScaler = preprocessing.MinMaxScaler()
scaled_x_train = minmaxScaler.fit_transform(X_train)
scaled_x_test = minmaxScaler.fit_transform(X_test)

In [39]:
# evaluation for ngrams and sentiment

# Encode labels from [-1, 1] to [0, 1]
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)

# create and train SVM model
xgboost_model = XGBClassifier()
xgboost_model.fit(scaled_x_train, y_train_enc)

# generate prediction
y_train_pred = le.inverse_transform(xgboost_model.predict(scaled_x_train))
y_test_pred = le.inverse_transform(xgboost_model.predict(scaled_x_test))

# display train and test accuracy
train_accuracy = accuracy_score(y_train, y_train_pred)
print('Training accuracy is {0}'.format(train_accuracy))
test_accuracy = accuracy_score(y_test, y_test_pred)
print('Testing accuracy is {0}'.format(test_accuracy))

# display precision
prec_score = precision_score(y_test, y_test_pred)
print('Precision: {0}'.format(prec_score))

# display recall
rec_score = recall_score(y_test, y_test_pred)
print('Recall: {0}'.format(rec_score))

Training accuracy is 0.9509408602150538
Testing accuracy is 0.6324043887147336
Precision: 0.6317829457364341
Recall: 0.6044117647058823
